In [41]:
import json

########################################################################

# Do not install any external packages. You can only use Python's default libraries such as:
# json, math, itertools, collections, functools, random, heapq, etc.

########################################################################




class Inference:
    def __init__(self, data):
        """
        Initialize the Inference class with the input data.
        
        Parameters:
        -----------
        data : dict
            The input data containing the graphical model details, such as variables, cliques, potentials, and k value.
        
        What to do here:
        ----------------
        - Parse the input data and store necessary attributes (e.g., variables, cliques, potentials, k value).
        - Initialize any data structures required for triangulation, junction tree creation, and message passing.
        
        Refer to the sample test case for the structure of the input data.
        """
        self.VariablesCount = data["VariablesCount"]
        self.Potentials_count = data["Potentials_count"]
        self.cliques = []
        self.potentials = []
        for clique in data["Cliques and Potentials"]:
            self.cliques.append(clique["cliques"])
            self.potentials.append(clique["potentials"])

        self.neighbours = [[] for _ in range(self.VariablesCount)]
        for node in range(self.VariablesCount):
            for clique in self.cliques:
                if node in clique:
                   temp_clique = clique[:]
                   temp_clique.remove(node)
                   union_list = list(set(self.neighbours[node]) | set(temp_clique))
                   self.neighbours[node] = union_list
                   
        self.k_value = data["k value (in top k)"]
        pass

    def triangulate_and_get_cliques(self):
        """
        Triangulate the undirected graph and extract the maximal cliques.
        
        What to do here:
        ----------------
        - Implement the triangulation algorithm to make the graph chordal.
        - Extract the maximal cliques from the triangulated graph.
        - Store the cliques for later use in junction tree creation.

        Refer to the problem statement for details on triangulation and clique extraction.
        """
        # functions for the tasks:
        # Checking if a node is simplicial
        def simplicial_node(node_idx, neighbour_set):
            result = True
            for neighbour in neighbour_set[node_idx]:
                temp_neigh = neighbour_set[node_idx][:]
                temp_neigh.remove(neighbour)
                if(not(set(temp_neigh).issubset(set(neighbour_set[neighbour])))):
                   result = False
                   break
            return result if neighbour_set[node_idx] != [] else False
        
        self.elimination_ordering = []
        def triangulate(neighbour_set):
            neighbour_set_trimmed = [s.copy() for s in neighbour_set]
            
            newly_connected_nodes = []
            added_edges = []

            # STEP1: Remove simplicial nodes iteratively
            while(neighbour_set_trimmed != [[] for _ in range(len(neighbour_set_trimmed))]):
                while(any([simplicial_node(node, neighbour_set_trimmed) for node in range(len(neighbour_set_trimmed))])):
                    for node in range(len(neighbour_set_trimmed)):
                        if(simplicial_node(node, neighbour_set_trimmed)):
                            neighbour_set_trimmed[node] = []
                            for set in neighbour_set_trimmed:
                                set.remove(node) if node in set else None
                            self.elimination_ordering.append(node)

                if(neighbour_set_trimmed == [[] for _ in range(len(neighbour_set_trimmed))]):
                   break
                
                # STEP1.1: If no simplicial node at some stage, make the lowest order node simplicial
                ## STEP1.1.1: Finding the lowest order node
                min_order_node = -1
                min_length = float('inf')
                for node in range(len(neighbour_set_trimmed)):
                    if((len(neighbour_set_trimmed[node]) > 0) and (len(neighbour_set_trimmed[node]) < min_length)):
                       min_order_node = node

                ## STEP1.1.2: Making the above node simplicial
                for neighbour in neighbour_set_trimmed[min_order_node]:
                    min_order_neighbours = neighbour_set_trimmed[min_order_node][:]
                    min_order_neighbours.remove(neighbour)
                    union_list = list({*neighbour_set_trimmed[neighbour], *min_order_neighbours})
                    neighbour_set_trimmed[neighbour] = union_list
                    added_edges.append(union_list[:])
                    newly_connected_nodes.append(neighbour)
            
            neighbour_set_triangulated = [s.copy() for s in neighbour_set]
            
            for i in range(len(newly_connected_nodes)):
                neighbour_set_triangulated[newly_connected_nodes[i]] = list({*neighbour_set_triangulated[newly_connected_nodes[i]], *added_edges[i]})
                
            return neighbour_set_triangulated

        neighbour_set_triangulated = triangulate(self.neighbours)

        # Extracting the maximal cliques from the triangulated graph using iterative simplicial node removal:
        maximal_cliques = []
        neighbour_set_reduced = [s.copy() for s in self.neighbours]
        while(neighbour_set_reduced != [[] for _ in range(len(neighbour_set_reduced))]):
              for node in range(len(neighbour_set_reduced)):
                  if(simplicial_node(node, neighbour_set_reduced)):
                     temp_neighs = neighbour_set_reduced[node][:]
                     temp_neighs.append(node)
                     neighbour_set_reduced[node] = []
                     [neighbour_set_reduced[node_iter].remove(node) for node_iter in range(len(neighbour_set_reduced)) if node in neighbour_set_reduced[node_iter]]
                     if(not(any([a for a in maximal_cliques if set(temp_neighs).issubset(set(a))]))):
                        maximal_cliques.append(temp_neighs)       

        print(f"Original neighbours: {self.neighbours}")
        print(f"Triangulated graph : {neighbour_set_triangulated}")   
        print(f"Maximal cliques    : {maximal_cliques}")
        print(f"Elimination ordering : {self.elimination_ordering}")
        
        self.neighbours_triangulated = neighbour_set_triangulated
        self.maximal_cliques = maximal_cliques
        
        pass

    def get_junction_tree(self):
        """
        Construct the junction tree from the maximal cliques.
        
        What to do here:
        ----------------
        - Create a junction tree using the maximal cliques obtained from the triangulated graph.
        - Ensure the junction tree satisfies the running intersection property.
        - Store the junction tree for later use in message passing.

        Refer to the problem statement for details on junction tree construction.
        """
        # self.junction_tree = [[] for _ in range(len(self.maximal_cliques))]
        self.junction_tree = []
        for i in range(len(self.elimination_ordering)):
                node = self.elimination_ordering[i]
                new_node = list({item for clique in self.maximal_cliques for item in clique if node in clique})
                # print(new_node)
                # new_node.remove(self.elimination_ordering[i-1]) if i > 0 else None
                self.junction_tree.append(new_node)
        
        print(f"Junction tree: {self.junction_tree}")

        pass

    def assign_potentials_to_cliques(self):
        """
        Assign potentials to the cliques in the junction tree.
        
        What to do here:
        ----------------
        - Map the given potentials (from the input data) to the corresponding cliques in the junction tree.
        - Ensure the potentials are correctly associated with the cliques for message passing.
        
        Refer to the sample test case for how potentials are associated with cliques.
        """
        pass

    def get_z_value(self):
        """
        Compute the partition function (Z value) of the graphical model.
        
        What to do here:
        ----------------
        - Implement the message passing algorithm to compute the partition function (Z value).
        - The Z value is the normalization constant for the probability distribution.
        
        Refer to the problem statement for details on computing the partition function.
        """
        pass

    def compute_marginals(self):
        """
        Compute the marginal probabilities for all variables in the graphical model.
        
        What to do here:
        ----------------
        - Use the message passing algorithm to compute the marginal probabilities for each variable.
        - Return the marginals as a list of lists, where each inner list contains the probabilities for a variable.
        
        Refer to the sample test case for the expected format of the marginals.
        """
        pass

    def compute_top_k(self):
        """
        Compute the top-k most probable assignments in the graphical model.
        
        What to do here:
        ----------------
        - Use the message passing algorithm to find the top-k assignments with the highest probabilities.
        - Return the assignments along with their probabilities in the specified format.
        
        Refer to the sample test case for the expected format of the top-k assignments.
        """
        pass

In [42]:

########################################################################

# Do not change anything below this line

########################################################################

class Get_Input_and_Check_Output:
    def __init__(self, file_name):
        with open(file_name, 'r') as file:
            self.data = json.load(file)
    
    def get_output(self):
        n = len(self.data)
        output = []
        for i in range(n):
            inference = Inference(self.data[i]['Input'])
            inference.triangulate_and_get_cliques()
            inference.get_junction_tree()
            inference.assign_potentials_to_cliques()
            z_value = inference.get_z_value()
            marginals = inference.compute_marginals()
            top_k_assignments = inference.compute_top_k()
            output.append({
                'Marginals': marginals,
                'Top_k_assignments': top_k_assignments,
                'Z_value' : z_value
            })
        self.output = output

    def write_output(self, file_name):
        with open(file_name, 'w') as file:
            json.dump(self.output, file, indent=4)


if __name__ == '__main__':
    evaluator = Get_Input_and_Check_Output('Sample_Testcase - Copy.json')
    evaluator.get_output()
    evaluator.write_output('Sample_Testcase_Output.json')

Original neighbours: [[1, 4, 5], [0, 4, 5], [8, 4, 6], [8, 5, 7], [0, 1, 2, 5, 6, 8], [0, 1, 3, 4, 7, 8], [8, 2, 4], [8, 3, 5], [2, 3, 4, 5, 6, 7]]
Triangulated graph : [[1, 4, 5], [0, 4, 5], [8, 4, 6], [8, 5, 7], [0, 1, 2, 5, 6, 8], [0, 1, 3, 4, 7, 8], [8, 2, 4], [8, 3, 5], [2, 3, 4, 5, 6, 7]]
Maximal cliques    : [[1, 4, 5, 0], [8, 4, 6, 2], [8, 5, 7, 3], [4, 5, 8]]
Elimination ordering : [0, 1, 2, 3, 6, 7, 8, 4]
Junction tree: [[0, 1, 4, 5], [0, 1, 4, 5], [8, 2, 4, 6], [8, 3, 5, 7], [8, 2, 4, 6], [8, 3, 5, 7], [2, 3, 4, 5, 6, 7, 8], [0, 1, 2, 4, 5, 6, 8]]
Original neighbours: [[1, 2], [0, 2, 3], [0, 1, 3, 4, 5], [1, 2, 4, 5], [2, 3, 5, 6, 7], [2, 3, 4, 6], [4, 5, 7], [4, 6]]
Triangulated graph : [[1, 2], [0, 2, 3], [0, 1, 3, 4, 5], [1, 2, 4, 5], [2, 3, 5, 6, 7], [2, 3, 4, 6], [4, 5, 7], [4, 6]]
Maximal cliques    : [[1, 2, 0], [2, 3, 1], [3, 4, 5, 2], [4, 6, 5], [4, 7, 6]]
Elimination ordering : [0, 1, 2, 3, 5, 6, 7]
Junction tree: [[0, 1, 2], [0, 1, 2, 3], [0, 1, 2, 3, 4, 5], [1, 2